In [ ]:
# !pip install openai
# !pip install fuzzywuzzy

In [ ]:
!python --version

In [ ]:
import json
import random

import numpy as np
import openai
import pandas as pd

openai.api_key = "sk-WYfx2w0AwDkwBxyixWfl77Cy2VYhOgWvQOlQbVqc"

In [ ]:
STATUS_CHECK = "status_check"
FEES_INQUIRY = "fees_inquiry"
GET_CREDENTIALS = "get_credentials"
CHANGE_CREDENTIALS = "change_credential_info"
ACCOUNT_ACTIVITY = "get_account_activity"
ACCOUNT_BALANCE = "get_account_balance"
# -------------------------------------------
DUES_CHECK = "dues"
ACTIVATE = "activate"
REQUEST_CARD_ACTION = "replace_cancel_report_missing_card"
REQUEST_INFO_INTENT = "ask_for_info"
APPLY_FOR = "apply_for"

In [ ]:
def response_parser(openai_response: object):
    """
    response: OpenAI object response
    """
    response = json.loads(str(openai_response))["choices"][0]["text"]
    keyword = "Keywords: "
    text = "Text: "

    parsed_response = []
    for entries in response.strip().split("\n\n")[1:]:
        entry = entries.split("\n")
        keywords_string = entry[0].strip().replace(keyword, "")
        parsed_entry = {
            "keywords": keywords_string.split(","),
            "sentence": entry[1].strip().replace(text, "").replace("Text:", ""),
        }
        parsed_response.append(parsed_entry)
    return parsed_response

In [ ]:
configurations = [
    {
        "max_tokens": 300,
        "temperature": 0.8,
        "presence_penalty": 0.3,
        "frequency_penalty": 0.6,
    },
    {
        "max_tokens": 300,
        "temperature": 0.75,
        "presence_penalty": 0.6,
        "frequency_penalty": 0.4,
    },
    {
        "max_tokens": 300,
        "temperature": 0.65,
        "presence_penalty": 0.4,
        "frequency_penalty": 0.2,
    },
    {
        "max_tokens": 300,
        "temperature": 0.7,
        "top_p": 0.9,
        "frequency_penalty": 0.5,
    },
]
configurations.extend(
    [
        {
            "max_tokens": 300,
            "top_p": round(random.uniform(0.7, 0.98), 3),
            "presence_penalty": round(random.uniform(0.1, 0.4), 3),
            "frequency_penalty": round(random.uniform(0.3, 0.6), 3),
        }
        for x in range(3)
    ]
)
configurations

In [ ]:
prompts = {
    STATUS_CHECK: {
        "prompt": """
Keywords: savings account, disable, check status
Text: Could you please check if my savings account has been disabled on not? I had asked the agent yesterday to disable my savings account.

Keywords: visa, credit card, activation, status
Text: What is the status of my visa card? My new credit card was supposed to be activated by today, but is still not.

Keywords: home loan, apply, status
Text: I had applied for a home last month. Why is there no status update? Can you please tell me the status of my application?
""",
        "type2": [
            "I",
            "we",
            "disabled",
            "blocked",
            "turn off",
            "cancelled",
            "suspended",
            "activated",
            "turn on",
        ],
        "type3": [
            "status",
            "check status",
            "status update",
            "any update",
        ],
    },
    FEES_INQUIRY: {
        "prompt": """
Keywords: savings account, charges, question
Text: my question is how much is the annual charges for a savings account? Is it cheap?

Keywords: visa, credit card, inquire, yearly fee
Text: What is the yearly fee on my visa card? Is there a maintenance charge or is it free? Any limits that I should know

Keywords: home loan, charges
Text: My loan got approved last week. I have not been told what the fees are for the sanction. Are there any hidden charges I should know of? 
""",
        "type2": [
            "charges",
            "fees",
            "amount",
            "transaction fee",
            "banking charges",
            "annual fee",
            "mothly maintenance charge",
        ],
        "type3": [
            "query",
            "question",
            "inquire",
            "inquiry",
            "doubt",
            "confused",
            "no clue",
        ],
    },
    GET_CREDENTIALS: {
        "prompt": """
Keywords: savings account, email, get
Text: Could you get my registered email id. I want to see which one I'm using for my savings account at your bank.

Keywords: visa, credit card, mobile, display
Text: What mobile number is connected to my visa credit card? can you display it?

Keywords: home loan, address, display, verify
Text: Can you display the home address you have on file for the loan application. I want to verify you have correct information.
""",
        "type2": [
            "email id",
            "mail id",
            "email",
            "mail",
            "phone",
            "phone number",
            "contact number",
            "mobile number",
            "mob no",
            "registered email",
            "sign in detail",
            "account number",
            "account no",
            "a/c no",
            "address",
            "mob num",
            "contact num" "registered phone number",
        ],
        "type3": ["display", "show", "check", "search", "verify", "tell", "assist"],
    },
    CHANGE_CREDENTIALS: {
        "prompt": """
Keywords: savings account, correct, Date Of Birth
Text: I just checked my account details, my birthdate is wrong there. Please correct my DOB!

Keywords: visa, credit card, number, change
Text: Is it possible to change the number on my visa card. I dont like credit cards that end with 4.

Keywords: home loan, contact, modify
Text: I had taken a home loan from you last year. I have changed my contact details, and want to update it in your records. How do I do that?
""",
        "type2": [
            "email id",
            "mail id",
            "email",
            "mail",
            "phone",
            "phone number",
            "contact number",
            "mobile number",
            "mob no",
            "registered email",
            "sign in detail",
            "account number",
            "account no",
            "a/c no",
            "address",
            "mob num",
            "contact num" "registered phone number",
        ],
        "type3": [
            "change",
            "modify",
            "please modify",
            "update",
            "help me to update",
            "correct",
            "help me to correct",
            "reset",
            "forgot",
            "modify",
        ],
    },
    ACCOUNT_ACTIVITY: {
        "prompt": """
Keywords: savings account, statement, download
Text: How to dowload the savings account activity statement for this month.

Keywords: transaction, business account, display
Text: Can you tell me how to display the account transactions done on my business account. I cant find it in the website.

Keywords: amex card, summary, check
Text: The amex card statement did not come to my email this month. Please tell me where I can check my transactions and summary for last quarter
""",
        "type2": ["download", "view", "display", "when was", "get", "where", "search"],
        "type3": [
            "account summary",
            "statement",
            "transactions",
            "credit information",
            "summary",
            "activity",
            "account activity",
            "transaction statement",
        ],
    },
    ACCOUNT_BALANCE: {
        "prompt": """
Keywords: savings account, display, balance
Text: Could you please check if my savings account has enough balance. Please display me the savings account balance.

Keywords: visa, debit card, money, status, account
Text: How to check the money available on my visa debit card? Could you show me the current amount?

Keywords: demat account, balance, assist,
Text: I have a demat account that I have not used in some time. Where can I see the balance in the acc. Please assist me to find it/
""",
        "type2": ["display", "show me", "check", "where to check", "where to see"],
        "type3": [
            "account",
            "balance",
            "amount",
            "figure",
            "money",
            "bal",
            "acc balance",
            "account balance",
        ],
    },
}

In [ ]:
ents = [
    "savings",
    "salary",
    "ADIB",
    "Ghina",
    "current",
    "recurring deposit",
    "fixed deposit",
    "loan",
    "checking",
    "corporate",
    "card",
    "covered card",
    "credit card",
    "debit card",
    "atm card",
    "amex",
    "diners",
    "jcb",
    "maestro",
    "mastercard",
    "visa",
]

In [ ]:
for intent_key, intent in prompts.items():
    parsed_responses = []
    unparsed_responses = []
    prompt = intent["prompt"]
    for i in range(100):
        engine = random.choice(["davinci", "curie"])
        sample_keys = (
            random.sample(ents, 2)
            + random.sample(intent["type2"], 1)
            + random.sample(intent["type3"], 1)
        )
        keys = ", ".join(sample_keys)

        for config in configurations:
            response = openai.Completion.create(
                engine=engine,
                prompt=f"{prompt}\n Keywords: {keys}\n Text: ",
                **config,
            )
            try:
                parsed = response_parser(response)
            except:
                unparsed_responses.append(response)
            else:
                #         print(parsed)
                parsed_responses.extend(parsed)
    with open(f"../data/test_parsed_{intent_key}.json", "w") as f:
        json.dump(parsed_responses, f, indent=2)
    with open(f"../data/test_unparsed_{intent_key}.json", "w") as f:
        json.dump(unparsed_responses, f, indent=2)

In [ ]:
import glob

unparsed_paths = glob.glob('../data/generated_raw_data/unparsed/test_unpa*')
unparsed_paths

In [ ]:
def unparsed_parser(file):
    sentences = [sentence['choices'][0]['text'].strip('\n').strip().replace('\n\n','\n').split('\n') for sentence in file]
    is_key=False
    parsed = []
    unparsed = []
    temp = dict()
    for sentence in sentences:
        for s in sentence:
            if s.startswith('Keywords:'):
                if is_key==True:
                    unparsed.append(temp)
                    temp = dict()

                s = s.replace('Keywords:','').strip()
                temp['keywords'] = s
                is_key=True

            elif s.startswith('Text:'):
                s = s.replace('Text:','').strip()
                temp['sentence'] = s

                if 'keywords' in temp:
                    if temp['sentence'] == "":
                        temp = dict()
                        is_key=False
                        continue
                        
                    parsed.append(temp)
                else:
                    unparsed.append(temp)

                temp = dict()
                is_key=False
    return parsed, unparsed

In [ ]:
unparsed = []
parsed = []
for unp_path in unparsed_paths:
    with open(unp_path) as f:
        file = json.load(f)
    temp_parsed, temp_unparsed = unparsed_parser(file)    
    unparsed.extend(temp_unparsed)
    parsed.extend(temp_parsed)
with open('../data/generated_raw_data/parsed//parsed_round2.json', 'w') as f:
    json.dump(parsed, f, indent=2)
with open('../data/generated_raw_data/unparsed/unparsed_round2.json', 'w') as f:
    json.dump(unparsed, f, indent=2)
    